requires_grad=True PyTorch 会记录它的计算图并在 .grad 中保存梯度

loss.backward() 所有可训练参数的 .grad 被填充

optimizer.step() 对应的是 param.data -= lr * param.grad 手动更新权重这句话

requires_grad=False 冻结这层参数（不求导，不更新）

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义一个三层模型
class TinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))   # layer1
        x = F.relu(self.fc2(x))   # layer2
        x = self.fc3(x)           # layer3
        return x

net = TinyNet()

# 冻结中间层 fc2
for p in net.fc2.parameters():
    p.requires_grad = False

# 输入与目标
x = torch.tensor([[1.0, 2.0]])
y_true = torch.tensor([[1.0]])

# 前向
y_pred = net(x)
loss = F.mse_loss(y_pred, y_true)
loss.backward()

# 查看梯度
for name, p in net.named_parameters():
    print(name, p.requires_grad, p.grad)


fc1.weight True tensor([[0.0000, 0.0000],
        [0.0031, 0.0063]])
fc1.bias True tensor([0.0000, 0.0031])
fc2.weight False None
fc2.bias False None
fc3.weight True tensor([[-3.3135,  0.0000]])
fc3.bias True tensor([-2.5688])
